# ResizeMix

In [1]:
#| default_exp resizemix

In [2]:
#| export
# library
from semantic_segmentation_augmentations.holemakertechnique import HoleMakerTechnique, HoleMakerBounded
from semantic_segmentation_augmentations.regionmodifier import RegionModifier
from semantic_segmentation_augmentations.iholesfilling import HolesFilling

# others
import cv2
import random
from fastai.basics import *
import numpy as np

/home/ruescog/.conda/envs/visionmodelsevaluation/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
#| hide
from nbdev.showdoc import *

In [4]:
#| export
class ResizeMix(HolesFilling):
    "Defines the amount of holes, the technique used to make them and the probability of apply the technique."
    def __init__(self,
                 holes_num = 1, # The amount of holes to make.
                 modifier: "RegionModifier" = None, # The modifier that defines the traditional augments to apply to the selected regions.
                 hole_maker: "HoleMakerTechnique" = None, # The strategy used to make the holes.
                 p = 0.5): # The probability of applying this technique.
        hole_maker = hole_maker if hole_maker else HoleMakerBounded(hole_size = (64, 64))
        super().__init__(modifier, hole_maker)
        self.holes_num = holes_num
        self.p = p

    def before_batch(self):
        "Applies the CutMixResizeMix technique (fills a hole with the whole resized image)."
        
        if not self.training:
            return
        
        for image, mask in zip(self.x, self.y):
            if random.random() < self.p:
                for _ in range(self.holes_num):
                    xhole, yhole = self.make_hole(mask)
                    # It is needed to permute the image because opencv works with a (H,W,C) format
                    sub_image, sub_mask = image.permute(1, 2, 0), mask.transpose(1, 0)
                    hole_size = self.hole_maker.hole_size
                    sub_image = TensorBase(cv2.resize(np.array(sub_image.cpu()), hole_size))
                    # It is needed to interpolate the integers without transforming them into floats
                    sub_mask = TensorBase(cv2.resize(np.array(sub_mask.cpu()), hole_size, interpolation = cv2.INTER_LINEAR_EXACT))
                    sub_image, sub_mask = sub_image.permute(2, 0, 1), sub_mask.transpose(1, 0)
                    self.fill_hole(image, mask, xhole, yhole, [sub_image, sub_mask])

The default technique used to make those holes is the `HoleMakerBounded` technique.

`ResizeMix` was defined and implemented [here](https://doi.org/10.48550/arXiv.2012.11101).

In [5]:
show_doc(ResizeMix.before_batch)

---

[source](https://github.com/ruescog/semantic_segmentation_augmentations/blob/master/semantic_segmentation_augmentations/resizemix.py#L31){target="_blank" style="float:right; font-size:smaller"}

### ResizeMix.before_batch

>      ResizeMix.before_batch ()

Applies the CutMixResizeMix technique (fills a hole with the whole resized image).

In [6]:
#| hide
import nbdev
nbdev.nbdev_export()